In [1]:
from functools import cache
from pathlib import Path

import chrysalis as chry
import duckdb
import polars as pl
from itables import show

import transformations as transformations
import invariants as invariants

In [2]:
@cache
def _test_connection() -> duckdb.DuckDBPyConnection:
    input_data_path = Path("./player_stats.parquet")
    if not input_data_path.exists():
        raise RuntimeError("Input dataset does not exist, exiting.")

    conn = duckdb.connect()
    conn.register(
        "player_stats",
        pl.read_parquet(input_data_path).cast({"round": pl.Int8}),
    )
    return conn


def evaluate_query(query: str) -> pl.DataFrame:
    conn = _test_connection()
    return conn.query(query).pl()


_TEST_QUERY = "SELECT name, position, college, team, round, draft FROM player_stats;"

In [3]:
chry.register(transformations.add_college_column, invariants.length_equals)
chry.register(transformations.add_team_column, invariants.length_equals)
chry.register(transformations.add_round_column, invariants.length_equals)
chry.register(transformations.add_draft_column, invariants.length_equals)
chry.register(transformations.remove_college_column, invariants.length_equals)
chry.register(transformations.remove_team_column, invariants.length_equals)
chry.register(transformations.remove_round_column, invariants.length_equals)
chry.register(transformations.remove_draft_column, invariants.length_equals)

chry.register(transformations.add_order_by_asc, invariants.length_equals)
chry.register(transformations.add_order_by_desc, invariants.length_equals)
chry.register(transformations.remove_order_by, invariants.length_equals)

# Intentional bug, it is possible that adding `LIMIT 400` will remove `LIMIT 200`
# and cause the invariant to fail.
chry.register(transformations.add_limit_400, invariants.length_less_than_equals)
chry.register(transformations.add_limit_200, invariants.length_less_than_equals)
chry.register(transformations.remove_limit, invariants.length_greater_than_equals)

In [5]:
conn = chry.run(evaluate_query, [_TEST_QUERY], chain_length=50, num_chains=10)

In [6]:
show(conn.query("SELECT COUNT(*) AS count FROM applied_transformation;").pl())

count
Loading ITables v2.3.0 from the internet... (need help?)


In [8]:
chain_start_ids = conn.query("""SELECT id FROM applied_transformation WHERE previous_applied_transformation IS NULL;""").pl()
show(chain_start_ids)

id
Loading ITables v2.3.0 from the internet... (need help?)


In [11]:
relation_chain = conn.execute("""
WITH RECURSIVE transformation_chain AS (
    SELECT 
        id,
        transformation,
        previous_applied_transformation,
        link_index,
        created_at
    FROM applied_transformation
    WHERE id = ?

    UNION ALL

    SELECT 
        at.id,
        at.transformation,
        at.previous_applied_transformation,
        at.link_index,
        at.created_at
    FROM applied_transformation at
    JOIN transformation_chain tc ON tc.id = at.previous_applied_transformation
)
SELECT tc.link_index, t.name, t.id FROM transformation_chain tc
INNER JOIN transformation t ON tc.transformation = t.id
ORDER BY link_index;
""", (chain_start_ids[1]["id"].item(),)).pl()
show(relation_chain)

In [12]:
query = _TEST_QUERY
knowledge_base = chry._internal._controller._CURRENT_KNOWLEDGE_BASE

print(f"{'start':<30} {query}")
for (_, name, transform_id) in relation_chain.head(10).iter_rows():
    query = knowledge_base.relations[transform_id].apply_transform(query)
    print(f"{name:<30} {query}")

start                          SELECT name, position, college, team, round, draft FROM player_stats;
add_team_column                SELECT name, position, college, round, draft, team FROM player_stats
add_limit_200                  SELECT name, position, college, round, draft, team FROM player_stats LIMIT 200
add_college_column             SELECT name, position, round, draft, team, college FROM player_stats LIMIT 200
add_college_column             SELECT name, position, round, draft, team, college FROM player_stats LIMIT 200
remove_team_column             SELECT name, position, round, draft, college FROM player_stats LIMIT 200
add_draft_column               SELECT name, position, round, college, draft FROM player_stats LIMIT 200
remove_limit                   SELECT name, position, round, college, draft FROM player_stats
add_order_by_desc              SELECT name, position, round, college, draft FROM player_stats ORDER BY draft DESC, round DESC
add_limit_200                  SELECT nam

In [20]:
show(conn.query("""
SELECT DISTINCT name FROM failed_invariant
INNER JOIN applied_transformation ON failed_invariant.applied_transformation = applied_transformation.id
INNER JOIN transformation ON applied_transformation.transformation = transformation.id
""").pl())

name
Loading ITables v2.3.0 from the internet... (need help?)
